In [2]:
import pandas as pd
import sys
import numpy as np
from numpy.linalg import inv
import json

sys.path.append('../')

import data_generation

In [23]:
with open('Examples_data/SP500.json', 'r') as f:
    stock_dict = json.load(f)

inverse_stock_dict = {value: key for key, value in stock_dict.items()}

data = pd.read_csv('Examples_data/finance.csv', header=None)  
data.columns = [inverse_stock_dict.get(i, f"Unnamed_{i}") for i in range(len(data.columns))] 

data.head()


,A,AA,AAPL,ABC,ABT,ACE,ACS,ADBE,ADI,ADM,...,XL,XLNX,XOM,XRAY,XRX,XTO,YHOO,YUM,ZION,ZMH
0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,-0.23,-1.15,-0.13,-1.22,1.49,0.0,0.58,-0.37,-0.67,-1.01,...,3.64,-0.44,1.94,0.14,1.27,-0.42,1.32,-2.45,-3.60,1.49
2,-0.94,-0.88,-0.11,0.31,-0.66,0.0,-0.56,-0.50,0.14,-2.61,...,2.86,-0.99,-1.19,0.32,1.55,-3.94,0.22,-0.26,-0.37,0.06
3,0.51,-0.77,-1.22,-0.19,0.28,0.0,-0.04,-0.53,0.91,0.53,...,0.13,0.05,0.93,-1.11,-0.04,-0.15,-0.97,-0.44,-0.84,0.81
4,-0.15,0.75,0.83,0.81,0.78,0.0,-1.51,-0.54,-0.56,0.72,...,-1.16,-0.79,-1.10,-0.97,0.99,-1.80,-0.39,-0.76,-0.27,-0.37


In [24]:
data.shape

(245, 524)